In [1]:
import os
import fitz
import re
import json
from PyPDF2 import PdfReader

In [2]:
def extract_text_from_pdf(pdf_path):
    reader = PdfReader(pdf_path)
    text = ""
    for page in reader.pages:
        text += page.extract_text() + "\n"
    return text

def parse_pdf_text(text, doc_title="Неизвестный документ"):
    result = {
        "документ": doc_title,
        "главы": []
    }

    # Разбиваем по главам: Глава I. ОБЩИЕ ПОЛОЖЕНИЯ
    главы_raw = re.split(r"\n?Глава\s+([IVXLCDM]+)\.\s+([^\n]+)", text)
    for i in range(1, len(главы_raw), 3):
        номер_главы = главы_raw[i] + "."
        заголовок = главы_raw[i+1].strip()
        текст_главы = главы_raw[i+2]

        # Ищем статьи в каждой главе
        статьи = re.findall(r"Статья\s+(\d+)\.\s+([^\n]+)(.*?)(?=Статья\s+\d+\.|\Z)", текст_главы, re.DOTALL)

        пункты = []
        for номер, заголовок_статьи, тело in статьи:
            тело = re.sub(r"\s+", " ", тело.strip())
            текст = f"{заголовок_статьи.strip()} {тело}".strip()
            пункты.append({
                "номер": f"{номер}.",
                "текст": текст
            })

        result["главы"].append({
            "глава": номер_главы,
            "заголовок": заголовок,
            "пункты": пункты
        })

    return result

In [3]:
pdf_path = "C:\\Users\\My Computer\\Desktop\\Work\\Learn\\OPK_Project\\documents_pdf\\19_03_1997_N_60_FZ.pdf"
outp_path = "C:\\Users\\My Computer\\Desktop\\Work\\Learn\\OPK_Project\\documents_json\\19_03_1997_N_60_FZ.json"
document_name = "Воздушный кодекс Российской Федерации от 19.03.1997 N 60-ФЗ (в ред. от 08.08.2024)"

text = extract_text_from_pdf(pdf_path)
parsed_json = parse_pdf_text(text, doc_title=document_name)

with open(outp_path, "w", encoding="utf-8") as f:
    json.dump(parsed_json, f, ensure_ascii=False, indent=2)